## SRJ

## Adapt - GCIM

## First step - To create Generating functions

## Generating fns code ready.  single & 2 singles

## Gradients added and state is ready

In [1]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/4

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
  


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    excitationlist.extend(singles)
    print('Excitations are', excitationlist)
    #print('Going to print the single generating fns.')
    for excitation in excitationlist:
        @qml.qnode(dev)
        def superposition1():
            #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
            qml.StatePrep(hf_state, wires=range(qubits))
            #print('Single Exc going in', excitation)
            qml.SingleExcitation(theta, wires=excitation)
            return qml.state()
        super1 = superposition1()
        generatingfns.append(super1)
        #print('S-Generating functions are', super1)
    #print('After adding single gfs, len is', len(generatingfns))
    print('\n\n\n')
    #print('Printing 2s generating fns')
    for i, j in itertools.combinations(excitationlist, 2):
        if len(i) == 2 and len(j) == 2:  # Ensure both are singles
            #print(f'Applying excitations: {i} and {j}')
            @qml.qnode(dev)
            def superposition2():
                #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
                qml.StatePrep(hf_state, wires=range(qubits))
                qml.SingleExcitation(theta, wires=i)
                qml.SingleExcitation(theta, wires=j)
                return qml.state()                
            super2= superposition2()
            generatingfns.append(super2)
            #print('D-Generating functions are', super2)
    #print('Total len of Generating functions is', len(generatingfns))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]


    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
        
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                #print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            #print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                print('The value of i and j is', i, j)
                S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy)    
    return ash_excitation, states,eig,gs_energy


In [2]:
ash_excitation, states,eig,gs_energy = adaptvqe(adapt_it=6, e_th=1e-12)

HF state is -1.3325688880668918
Excitations are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]




The adapt iteration now is 0


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 0.31004515885821066 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
ash_excitation is [[2, 3, 6, 7]]
The length of ash_excitation before generating matrix is 1
0: ─╭|Ψ⟩───────────┤  State
1: ─├|Ψ⟩───────────┤  State
2: ─├|Ψ⟩─╭G²(0.79)─┤  State
3: ─├|Ψ⟩─├G²(0.79)─┤  State
4: ─├|Ψ⟩─│─────────┤  State
5: ─├|Ψ⟩─│─────────┤  State
6: ─├|Ψ⟩─├G²(0.79)─┤  State
7: ─╰|Ψ⟩─╰G²(0.79)─┤  State
State count after Generating function is {'11000011': 151, '11110000': 849}
After adding new state, the length of states is 2
The value of i and j is 0 0
The value of i and j is 0 1
The value of i and j is 1 0
The value of i and j is 1 1
S matrix with 1 and 1 is [[1.        +0.j 0.92387953-0.j]
 [0.92387953+0.j 1.        +0.j]]
Hamiltonian Matrix with 1 1 is [[-1.33256889+0.j -1.29045769-0.j]
 [-1.29045769+0.j -1.42037613+0.j]]
Eigenvalues are [-1.43008367 -1.0861242 ]
Ground state energy is [-1.430083670535935]
The adapt iteration now is 1


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 0.3658436432870543 for operator a⁺(0) a⁺(3) a(5) a(6)
Highest gradient excitation is [0, 3, 5, 6]
ash_excitation is [[2, 3, 6, 7], [0, 3, 5, 6]]
The length of ash_excitation before generating matrix is 2
0: ─╭|Ψ⟩───────────╭G²(0.79)─┤  State
1: ─├|Ψ⟩───────────│─────────┤  State
2: ─├|Ψ⟩─╭G²(0.79)─│─────────┤  State
3: ─├|Ψ⟩─├G²(0.79)─├G²(0.79)─┤  State
4: ─├|Ψ⟩─│─────────│─────────┤  State
5: ─├|Ψ⟩─│─────────├G²(0.79)─┤  State
6: ─├|Ψ⟩─├G²(0.79)─╰G²(0.79)─┤  State
7: ─╰|Ψ⟩─╰G²(0.79)───────────┤  State
State count after Generating function is {'01100110': 130, '11000011': 131, '11110000': 739}
After adding new state, the length of states is 3
The value of i and j is 0 0
The value of i and j is 0 1
The value of i and j is 0 2
The value of i and j is 1 0
The value of i and j is 1 1
The value of i and j is 1 2
The value of i and j is 2 0
The value of i and j is 2 1
The value of i and j is 2 2
S matrix with 2 and 2 is [[1.        +0.j 0.92387953-0.j 0.85355339

...

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 0.21111995508844042 for operator a⁺(0) a⁺(3) a(5) a(6)
Highest gradient excitation is [0, 3, 5, 6]
ash_excitation is [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 3, 5, 6]]
The length of ash_excitation before generating matrix is 5
0: ─╭|Ψ⟩───────────╭G²(0.79)─╭G²(0.79)───────────╭G²(0.79)─┤  State
1: ─├|Ψ⟩───────────│─────────├G²(0.79)─╭G²(0.79)─│─────────┤  State
2: ─├|Ψ⟩─╭G²(0.79)─│─────────│─────────├G²(0.79)─│─────────┤  State
3: ─├|Ψ⟩─├G²(0.79)─├G²(0.79)─│─────────│─────────├G²(0.79)─┤  State
4: ─├|Ψ⟩─│─────────│─────────├G²(0.79)─├G²(0.79)─│─────────┤  State
5: ─├|Ψ⟩─│─────────├G²(0.79)─╰G²(0.79)─│─────────├G²(0.79)─┤  State
6: ─├|Ψ⟩─├G²(0.79)─╰G²(0.79)───────────│─────────╰G²(0.79)─┤  State
7: ─╰|Ψ⟩─╰G²(0.79)─────────────────────╰G²(0.79)───────────┤  State
State count after Generating function is {'00001111': 145, '00111100': 112, '01100110': 260, '10011001': 26, '11000011': 125, '11110000': 332}
After adding new state, the length o

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 0.22815479832037758 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
ash_excitation is [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 3, 5, 6], [1, 2, 4, 7]]
The length of ash_excitation before generating matrix is 6
0: ─╭|Ψ⟩───────────╭G²(0.79)─╭G²(0.79)───────────╭G²(0.79)───────────┤  State
1: ─├|Ψ⟩───────────│─────────├G²(0.79)─╭G²(0.79)─│─────────╭G²(0.79)─┤  State
2: ─├|Ψ⟩─╭G²(0.79)─│─────────│─────────├G²(0.79)─│─────────├G²(0.79)─┤  State
3: ─├|Ψ⟩─├G²(0.79)─├G²(0.79)─│─────────│─────────├G²(0.79)─│─────────┤  State
4: ─├|Ψ⟩─│─────────│─────────├G²(0.79)─├G²(0.79)─│─────────├G²(0.79)─┤  State
5: ─├|Ψ⟩─│─────────├G²(0.79)─╰G²(0.79)─│─────────├G²(0.79)─│─────────┤  State
6: ─├|Ψ⟩─├G²(0.79)─╰G²(0.79)───────────│─────────╰G²(0.79)─│─────────┤  State
7: ─╰|Ψ⟩─╰G²(0.79)─────────────────────╰G²(0.79)───────────╰G²(0.79)─┤  State
State count after Generating function is {'00001111': 267, '00111100': 106, '01100110

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


In [ ]:
#ash_excitation_45 = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 3, 5, 6]]
#ash_excitation_90 = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [0, 3, 5, 6], [0, 3, 5, 6]]

## Measurement of shots

In [ ]:
# Measurement with shots
for i in range(len(generatingfns)):
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(generatingfns):
        qml.StatePrep(generatingfns[i], wires=range(qubits))
        return qml.counts()

    counts = measure(generatingfns)
    print(f"Counts for generating function {i}: {counts}")


Counts for generating function 0: {'01111000': 153, '11110000': 847}
Counts for generating function 1: {'01110010': 148, '11110000': 852}
Counts for generating function 2: {'10110100': 143, '11110000': 857}
Counts for generating function 3: {'10110001': 137, '11110000': 863}
Counts for generating function 4: {'11011000': 138, '11110000': 862}
Counts for generating function 5: {'11010010': 140, '11110000': 860}
Counts for generating function 6: {'11100100': 157, '11110000': 843}
Counts for generating function 7: {'11100001': 152, '11110000': 848}
Counts for generating function 8: {'01110010': 118, '01111000': 130, '11110000': 752}
Counts for generating function 9: {'00111100': 21, '01111000': 107, '10110100': 121, '11110000': 751}
Counts for generating function 10: {'00111001': 20, '01111000': 128, '10110001': 132, '11110000': 720}
Counts for generating function 11: {'01111000': 149, '11011000': 133, '11110000': 718}
Counts for generating function 12: {'01011010': 20, '01111000': 125, '